In [12]:
# !pip install pydicom
# !pip install dicomweb_client
# !pip install tqdm

In [13]:
from dicomweb_client.api import DICOMwebClient
import pydicom
import os
import time
import datetime
from tqdm import tqdm
import csv
import random

url = 'http://localhost:8080/dcm4chee-arc/aets/DCM4CHEE/rs'

client = DICOMwebClient(url)

Funkcja obliczania długości odczytywania plików z bazy:

In [14]:
sample_size = 100
def get_average_read_time(patient_UID):
    # warmup reads (for the first time search time is larger for some reason - idk why)
    client.search_for_studies(search_filters={'PatientID': str(patient_UID)})
    client.search_for_studies(search_filters={'PatientID': str(patient_UID)})
    
    samples = []
    for i in range(sample_size):
        samples.append(random.randint(0, patient_UID))
    
    start_time = time.time()
    
    for sample in samples:
        print(sample)
        client.search_for_studies(search_filters={'PatientID': str(sample)})
    
    search_time = (time.time() - start_time) / sample_size
    
    print("Patient UDI: " + str(patient_UID) + "; average search time: " + str(search_time) )
    return search_time

Przygotowanie pliku z logami:

In [15]:
path = "log/uploading_dicom_to_pacs_logs-" + str(time.strftime("%Y%m%d-%H%M%S")) + ".csv"
print("logs path: " + path)
header = ['timestamp', 'number_of_images', 'last_1k_dicoms_uploadtime', 'experiment_time', 'pacs_read_time']

with open(path, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

logs path: log/uploading_dicom_to_pacs_logs-20220510-222552.csv


Funkcja wgrywania paczki dicomów do pacsa:

In [16]:
def upload_1k_dicoms_into_pacs(images_prefix):
    start_time = time.time()
    
    for i in range(1000):
        uid = str(images_prefix) + str(i)
        
        dicom.SeriesInstanceUID = uid
        dicom.StudyInstanceUID = uid
        dicom.FrameOfReferenceUID = uid
        dicom.SOPInstanceUID = uid
        dicom.PatientID = uid
        client.store_instances(datasets=[dicom])
    
    return( time.time() - start_time )   

### Zapełnianie bazy dicomami bez obrazu i o prostym UID:

Przygotowanie dicoma:

In [17]:
dicom = pydicom.dcmread('test_images/IM-0014-0001.dcm')
dicom.PixelData = bytes()

Pętla wgrywania:

In [ ]:
uid_prefix = dicom.SeriesInstanceUID + '.'

experiment_start_time = time.time()

for i in tqdm(range(500)):
    upload_time = upload_1k_dicoms_into_pacs(uid_prefix + str(i))
    
    timestamp = datetime.datetime.now()
    number_of_images = i*1000
    last_1k_dicoms_uploadtime = upload_time
    experiment_time = time.time() - experiment_start_time
    pacs_read_time = get_average_read_time(number_of_images)
    
    data = [str(timestamp), str(number_of_images), str(last_1k_dicoms_uploadtime), str(experiment_time), str(pacs_read_time)]
    
    with open(path, 'a', encoding='UTF8', newline='') as f:
        csv.writer(f).writerow(data)
    

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]